In [1]:
import os
import pandas as pd
import geopandas as gpd
import rioxarray as riox

import rasterio

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

Open lidar and create auxiliary min, max, avg, diff lidar rasters

In [2]:
lidar_year = 2016 # must be 2016, 2018 or 2020

lidar_rast_r = rasterio.open(lsf.path_to_lidar(lidar_year))

lsf.save_min_max_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'temp'),
                              year = lidar_year)

lsf.save_avg_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'temp'),
                              year = lidar_year)


Add canopy heigh data from year `lidar_year` to points from all aois in year `aoi_year`.

In [3]:
aoi_year = 2016
lidar_year = 2016

# If aoi_year=2016, then use aois without point_conception (there are no 2016 polygons in that aoi)
aois = ['campus_lagoon','carpinteria','gaviota']
if aoi_year != 2016:
    aois.append('point_conception')

lidar_rast_r = rasterio.open(lsf.path_to_lidar(lidar_year))

In [6]:
for aoi in aois:
    pts_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
                         aoi+'_points',
                         aoi+'_points_'+str(aoi_year)+'.csv')

    ## Obtain CRS from itemid and create geodataframe from points
    itemid = pd.read_csv(pts_fp).naip_id[0]
    pts = lsf.geodataframe_from_csv(pts_fp, lsf.crs_from_itemid(itemid))

    ## Sample LIDAR
    pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)
    lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)
    
    # file paths to auxiliary LIDAR rasters
    lidar_fps = []
    for tag in ['maxs_', 'mins_', 'avgs_']:
        lidar_fps.append(os.path.join(os.getcwd(),
                                     'temp',
                                     'lidar_'+tag+ str(lidar_year)+'.tif'))

    ## Sample mmax, min and avg LIDAR
    maxs_rast_r = rasterio.open(lidar_fps[0])
    max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

    mins_rast_r = rasterio.open(lidar_fps[1])
    min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

    avg_rast_r = rasterio.open(lidar_fps[2])
    avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)
    
    ## Add all LIDAR data to pts dataframe
    pts['lidar'] = lidar_samples
    pts['max_lidar']= max_samples
    pts['min_lidar'] = min_samples
    pts['min_max_diff'] = pts.max_lidar - pts.min_lidar  # include difference
    pts['avg_lidar'] = avg_samples

    ## Save points with added LIDAR data
    ptslidar_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
                         aoi +'_points',
                         aoi +'_pts_spectral_lidar_'+str(aoi_year)+'.csv')
    pts.to_csv(ptslidar_fp)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Delete auxiliary LIDAR rasters created in the beginning 

In [7]:
for fp in lidar_fps:
    os.remove(fp)